In [98]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import datetime as dt
import pandas as pd
users=pd.read_csv('olist_customers_dataset.csv')
orders=pd.read_csv('olist_orders_dataset.csv')
order_items=pd.read_csv('olist_order_items_dataset.csv')

In [99]:
#подгрузил данные и необходимые библиотеки, далее смотрю данные

In [100]:
users.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [101]:
orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [102]:
order_items.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [103]:
#Сразу сджойним статусы заказов и продажи товаров для того, чтобы установить понятие "покупка"

In [104]:
items=order_items.merge(orders, how='inner',on='order_id')

In [105]:
items.dtypes

order_id                          object
order_item_id                      int64
product_id                        object
seller_id                         object
shipping_limit_date               object
price                            float64
freight_value                    float64
customer_id                       object
order_status                      object
order_purchase_timestamp          object
order_approved_at                 object
order_delivered_carrier_date      object
order_delivered_customer_date     object
order_estimated_delivery_date     object
dtype: object

In [106]:
#За покупку принято считать каждый товар в заказе, статус которого ="delivered" & order_approved_at !='NaN'(продажа,которыя принесла деньги)
#Без подтверждения оплаты считать покупку, на мой вгляд, некорректно, так как, вероятно продажа отпечаталсь некорректно по как либо причинам.
#Причины могут быть различные: от системного бага до каких либо мошеннечиеских действий(версии требуют проверки)
#Каждый товар считается покупкой ввиду того, что, во-первых: каждый товар в заказе может быть от разных селлеров,
#во-вторых: на маркетплейсах идет тенденция к моноотправлениям, в частности это касается маркеплейса, цвет которого напоминает небо))
#В разрезе по каждому товару результат аналитики будет более точным

In [107]:
items.order_approved_at=items.order_approved_at.fillna(0)

In [108]:
items['order_pay_success']=items['order_approved_at'] !=0 #для дальнейшего отбора данных

In [109]:
full_data=items.merge(users, how='inner',on='customer_id')

In [110]:
full_data.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_pay_success,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,3ce436f183e68e07877b285a838db11a,delivered,2017-09-13 08:59:02,2017-09-13 09:45:35,2017-09-19 18:34:16,2017-09-20 23:43:48,2017-09-29 00:00:00,True,871766c5855e863f6eccc05f988b23cb,28013,campos dos goytacazes,RJ
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,f6dd3ec061db4e3987629fe6b26e5cce,delivered,2017-04-26 10:53:06,2017-04-26 11:05:13,2017-05-04 14:35:00,2017-05-12 16:04:24,2017-05-15 00:00:00,True,eb28e67c4c0b83846050ddfb8a35d051,15775,santa fe do sul,SP
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,6489ae5e4333f3693df5ad4372dab6d3,delivered,2018-01-14 14:33:31,2018-01-14 14:48:30,2018-01-16 12:36:48,2018-01-22 13:19:16,2018-02-05 00:00:00,True,3818d81c6709e39d06b2738a8d3a2474,35661,para de minas,MG
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,d4eb9395c8c0431ee92fce09860c5a06,delivered,2018-08-08 10:00:35,2018-08-08 10:10:18,2018-08-10 13:28:00,2018-08-14 13:32:39,2018-08-20 00:00:00,True,af861d436cfc08b2c2ddefd0ba074622,12952,atibaia,SP
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,58dbd0b2d70206bf40e62cd34e84d795,delivered,2017-02-04 13:57:51,2017-02-04 14:10:13,2017-02-16 09:46:09,2017-03-01 16:42:31,2017-03-17 00:00:00,True,64b576fb70d441e8f1b2d7d446e483c5,13226,varzea paulista,SP


In [111]:
purchases_by_users=full_data.query("order_status == 'delivered' and order_pay_success == True")\
    .groupby('customer_unique_id',as_index=False)\
    .agg({'product_id':'count'})\
    .sort_values('product_id', ascending=False)\
    .reset_index(drop=True)

purchases_by_users.head()

,customer_unique_id,product_id
0,c8460e4251689ba205045f3ea17884a1,24
1,4546caea018ad8c692964e3382debd19,21
2,c402f431464c72e27330a67f7b94d4fb,20
3,698e1cf81d01a3d389d96145f7fa6df8,20
4,0f5ac8d5c31de21d2f25e24be15bbffb,18


In [112]:
one_purchase=purchases_by_users.query('product_id == 1').count()
one_purchase[0]

81737

In [113]:
users_percent_one_purchase= (one_purchase[0] / full_data.shape[0] * 100).round(0)
users_percent_one_purchase

73.0

In [114]:
#Исходя из вышеприведенных данных видим, что большая часть пользователей совершили всего лишь одну покупку, что говорит о 
# невысокой лояльности пользоватлей к нашему маркетплейсу

In [115]:
#=================================================================================================================TASK2

In [116]:
cancelled_orders=full_data.query("order_status =='unavailable' or order_status =='canceled'").reset_index(drop=True)
cancelled_orders.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_pay_success,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,00310b0c75bb13015ec4d82d341865a4,1,c8e7c2ef329fcda4a233e7e2f8bb8b7d,a2deecd5398f5df4987110c80a1972a3,2018-08-17 15:04:25,39.90,15.38,0dad07848c618cc5a4679a1bfe1db8d2,canceled,2018-08-15 14:29:08,2018-08-15 15:04:25,NaN,NaN,2018-08-29 00:00:00,True,ec979208947bbba310f2ad8e50963725,31160,belo horizonte,MG
1,00ae7a8b4936674ebb701d4a23719a79,1,66f09a6de4f2ef63aaf5349b60a0e0c0,bf0d50a6410d487dc97d2baac0a8c0be,2018-05-15 15:51:32,27.90,9.26,a7e8a3cb55d9c9a536992c6465a46a3f,canceled,2018-05-09 14:14:06,2018-05-09 15:51:32,NaN,NaN,2018-05-17 00:00:00,True,f0b9f688a3b9151bc6b38f4b5b36ea44,6539,santana de parnaiba,SP
2,00ff0cf5583758e6964723e42f111bf4,1,bb1fe952bdc16ef8974d4ca11aec19b7,b17b679f4f5ce2e03ce6968c62648246,2017-04-14 01:03:29,154.90,15.25,e3626ed5d1f2e3a02d692ec948b25eeb,canceled,2017-04-10 00:45:56,2017-04-10 01:03:29,NaN,NaN,2017-05-04 00:00:00,True,b9badb100ff8ecc16a403111209e3a06,88025,florianopolis,SC
3,013e9c654a339d80b53513da3c1ea437,1,bcac2cdcd851929a5caa3e1508d5abc9,0adac9fbd9a2b63cccaac4f8756c1ca8,2017-06-22 13:55:18,109.90,13.11,9391729f79cd9e5fd7ca884ca030f579,canceled,2017-06-18 13:48:48,2017-06-18 13:55:18,NaN,NaN,2017-07-07 00:00:00,True,05a3b50193563913613acada5c86b1d2,18710,arandu,SP
4,0148d3df00cebda592d4e5f966e300cc,1,ed08ea04c92f5f434c2362f7310fb328,0adac9fbd9a2b63cccaac4f8756c1ca8,2017-08-24 19:24:32,5.53,11.85,672510553bcfbf72de694db2cd88f046,canceled,2017-08-19 19:08:26,2017-08-19 19:24:32,NaN,NaN,2017-09-11 00:00:00,True,e75fd09539dff5c7c80a12781404ffea,4078,sao paulo,SP


In [117]:
cancelled_orders.shape

(549, 19)

In [118]:
cancelled_orders.isna().sum()


order_id                           0
order_item_id                      0
product_id                         0
seller_id                          0
shipping_limit_date                0
price                              0
freight_value                      0
customer_id                        0
order_status                       0
order_purchase_timestamp           0
order_approved_at                  0
order_delivered_carrier_date     473
order_delivered_customer_date    542
order_estimated_delivery_date      0
order_pay_success                  0
customer_unique_id                 0
customer_zip_code_prefix           0
customer_city                      0
customer_state                     0
dtype: int64

In [119]:
#7 заказов были отменены после того(или во время),как были доставлены(об этом говорит разница между общим количестовом 
#недосталвенных заказов) и количеством пустых строк в столбце "дата доставки заказа клиенту"

In [120]:
#заказ становится заказом(начинается его сборка),когда он подтвержден,поэтому за дату заказа принято считать столбец "order_approved_at"

In [121]:
cancelled_orders.order_approved_at=cancelled_orders.order_approved_at.apply(lambda x: (x.split(' ')[0]).split('-')[1])

In [122]:
#корректировка даты, для дальнейшей группировки

In [123]:
cancelled_orders.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_pay_success,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,00310b0c75bb13015ec4d82d341865a4,1,c8e7c2ef329fcda4a233e7e2f8bb8b7d,a2deecd5398f5df4987110c80a1972a3,2018-08-17 15:04:25,39.90,15.38,0dad07848c618cc5a4679a1bfe1db8d2,canceled,2018-08-15 14:29:08,08,NaN,NaN,2018-08-29 00:00:00,True,ec979208947bbba310f2ad8e50963725,31160,belo horizonte,MG
1,00ae7a8b4936674ebb701d4a23719a79,1,66f09a6de4f2ef63aaf5349b60a0e0c0,bf0d50a6410d487dc97d2baac0a8c0be,2018-05-15 15:51:32,27.90,9.26,a7e8a3cb55d9c9a536992c6465a46a3f,canceled,2018-05-09 14:14:06,05,NaN,NaN,2018-05-17 00:00:00,True,f0b9f688a3b9151bc6b38f4b5b36ea44,6539,santana de parnaiba,SP
2,00ff0cf5583758e6964723e42f111bf4,1,bb1fe952bdc16ef8974d4ca11aec19b7,b17b679f4f5ce2e03ce6968c62648246,2017-04-14 01:03:29,154.90,15.25,e3626ed5d1f2e3a02d692ec948b25eeb,canceled,2017-04-10 00:45:56,04,NaN,NaN,2017-05-04 00:00:00,True,b9badb100ff8ecc16a403111209e3a06,88025,florianopolis,SC
3,013e9c654a339d80b53513da3c1ea437,1,bcac2cdcd851929a5caa3e1508d5abc9,0adac9fbd9a2b63cccaac4f8756c1ca8,2017-06-22 13:55:18,109.90,13.11,9391729f79cd9e5fd7ca884ca030f579,canceled,2017-06-18 13:48:48,06,NaN,NaN,2017-07-07 00:00:00,True,05a3b50193563913613acada5c86b1d2,18710,arandu,SP
4,0148d3df00cebda592d4e5f966e300cc,1,ed08ea04c92f5f434c2362f7310fb328,0adac9fbd9a2b63cccaac4f8756c1ca8,2017-08-24 19:24:32,5.53,11.85,672510553bcfbf72de694db2cd88f046,canceled,2017-08-19 19:08:26,08,NaN,NaN,2017-09-11 00:00:00,True,e75fd09539dff5c7c80a12781404ffea,4078,sao paulo,SP


In [124]:
cancelled_per_month= cancelled_orders.groupby(['order_approved_at','order_status'],as_index=False)\
    .agg({'product_id':'count'})
cancelled_per_month

,order_approved_at,order_status,product_id
0,01,canceled,36
1,02,canceled,91
2,03,canceled,47
3,04,canceled,40
4,05,canceled,57
5,06,canceled,32
6,07,canceled,78
7,08,canceled,61
8,09,canceled,16
9,10,canceled,37


In [125]:
cancelled_per_month_mean=cancelled_per_month.product_id.mean().round(0)
cancelled_per_month_mean

42.0

In [126]:
#Среднее количество отмененных заказов 42, далее выведена детализация по причнам отмены, большинство заказов отменено пользователями

In [127]:
cancelled_per_month.pivot(index='order_status', columns='order_approved_at', values='product_id').fillna(0)

order_approved_at,01,02,03,04,05,06,07,08,09,10,11,12
order_status,,,,,,,,,,,,
canceled,36.0,91.0,47.0,40.0,57.0,32.0,78.0,61.0,16.0,37.0,38.0,9.0
unavailable,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0


In [128]:
#=================================================================================================================TASK3

In [129]:
full_data.order_approved_at=pd.to_datetime(full_data.order_approved_at)

In [130]:
full_data['weekday']=full_data.order_approved_at.dt.strftime('%A')

In [131]:
full_data.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_pay_success,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,weekday
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,3ce436f183e68e07877b285a838db11a,delivered,2017-09-13 08:59:02,2017-09-13 09:45:35,2017-09-19 18:34:16,2017-09-20 23:43:48,2017-09-29 00:00:00,True,871766c5855e863f6eccc05f988b23cb,28013,campos dos goytacazes,RJ,Wednesday
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,f6dd3ec061db4e3987629fe6b26e5cce,delivered,2017-04-26 10:53:06,2017-04-26 11:05:13,2017-05-04 14:35:00,2017-05-12 16:04:24,2017-05-15 00:00:00,True,eb28e67c4c0b83846050ddfb8a35d051,15775,santa fe do sul,SP,Wednesday
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,6489ae5e4333f3693df5ad4372dab6d3,delivered,2018-01-14 14:33:31,2018-01-14 14:48:30,2018-01-16 12:36:48,2018-01-22 13:19:16,2018-02-05 00:00:00,True,3818d81c6709e39d06b2738a8d3a2474,35661,para de minas,MG,Sunday
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,d4eb9395c8c0431ee92fce09860c5a06,delivered,2018-08-08 10:00:35,2018-08-08 10:10:18,2018-08-10 13:28:00,2018-08-14 13:32:39,2018-08-20 00:00:00,True,af861d436cfc08b2c2ddefd0ba074622,12952,atibaia,SP,Wednesday
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,58dbd0b2d70206bf40e62cd34e84d795,delivered,2017-02-04 13:57:51,2017-02-04 14:10:13,2017-02-16 09:46:09,2017-03-01 16:42:31,2017-03-17 00:00:00,True,64b576fb70d441e8f1b2d7d446e483c5,13226,varzea paulista,SP,Saturday


In [132]:
purchases_by_weekday= full_data.groupby(['product_id','weekday'])\
    .agg({'product_id':'count'}).rename(columns={'product_id':'quantity'}).reset_index()\
    .sort_values('quantity',ascending=False).drop_duplicates(subset='product_id').drop('quantity', axis=1)


purchases_by_weekday

,product_id,weekday
41087,aca2eb7d00ea1a7b8ebd4e68314663af,Tuesday
20156,53b36df67ebb7c41585e8d54d6772e08,Tuesday
15936,422879e10f46682990de24d770e7f83d,Tuesday
36579,99a4788cb24856965c36a24e339b6058,Tuesday
13063,368c6c730842d78016ad823897a372db,Thursday
...,...,...
23981,63dfa8b12200610ed7b5e41d3c888a25,Saturday
23996,63f52009dc3ddb733394e80183aed0f0,Saturday
23985,63e51e65ec59b56039c6427eed73dcf0,Tuesday
23986,63eb97106bfe47fe857edcf78f3e3e08,Saturday


In [133]:

#=================================================================================================================TASK4

In [134]:
success_purchases=full_data.query("order_status =='delivered' or order_pay_success == True").reset_index(drop=True)

In [135]:
success_purchases.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_pay_success,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,weekday
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,3ce436f183e68e07877b285a838db11a,delivered,2017-09-13 08:59:02,2017-09-13 09:45:35,2017-09-19 18:34:16,2017-09-20 23:43:48,2017-09-29 00:00:00,True,871766c5855e863f6eccc05f988b23cb,28013,campos dos goytacazes,RJ,Wednesday
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,f6dd3ec061db4e3987629fe6b26e5cce,delivered,2017-04-26 10:53:06,2017-04-26 11:05:13,2017-05-04 14:35:00,2017-05-12 16:04:24,2017-05-15 00:00:00,True,eb28e67c4c0b83846050ddfb8a35d051,15775,santa fe do sul,SP,Wednesday
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,6489ae5e4333f3693df5ad4372dab6d3,delivered,2018-01-14 14:33:31,2018-01-14 14:48:30,2018-01-16 12:36:48,2018-01-22 13:19:16,2018-02-05 00:00:00,True,3818d81c6709e39d06b2738a8d3a2474,35661,para de minas,MG,Sunday
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,d4eb9395c8c0431ee92fce09860c5a06,delivered,2018-08-08 10:00:35,2018-08-08 10:10:18,2018-08-10 13:28:00,2018-08-14 13:32:39,2018-08-20 00:00:00,True,af861d436cfc08b2c2ddefd0ba074622,12952,atibaia,SP,Wednesday
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,58dbd0b2d70206bf40e62cd34e84d795,delivered,2017-02-04 13:57:51,2017-02-04 14:10:13,2017-02-16 09:46:09,2017-03-01 16:42:31,2017-03-17 00:00:00,True,64b576fb70d441e8f1b2d7d446e483c5,13226,varzea paulista,SP,Saturday


In [136]:
success_purchases['month']=success_purchases.order_approved_at.dt.strftime('%B')

In [137]:
success_purchases['weeks_in_month']=success_purchases.order_approved_at.dt.daysinmonth/7

In [138]:
purchases_per_month=success_purchases.groupby(['customer_unique_id','month','weeks_in_month'])\
    .agg({'product_id':'count'}).reset_index().rename(columns={'product_id':'quantity'})
purchases_per_month.head(15)

,customer_unique_id,month,weeks_in_month,quantity
0,0000366f3b9a7992bf8c76cfdf3221e2,May,4.428571,1
1,0000b849f77a49e4a4ce2b2a4ca5be3f,May,4.428571,1
2,0000f46a3911fa3c0805444483337064,March,4.428571,1
3,0000f6ccb0745a6a4b88665a16c9f078,October,4.428571,1
4,0004aac84e0df4da2b147fca70cf8255,November,4.285714,1
5,0004bd2a26a76fe21f786e4fbd80607f,April,4.285714,1
6,00050ab1314c0e55a6ca13cf7181fecf,April,4.285714,1
7,00053a61a98854899e70ed204dd4bafe,February,4.000000,2
8,0005e1862207bf6ccc02e4228effd9a0,March,4.428571,1
9,0005ef4cd20d2893f0d9fbd94d3c0d97,March,4.428571,1


In [139]:
purchases_per_month['purchases_by_month']=purchases_per_month.quantity / purchases_per_month.weeks_in_month
purchases_per_month.head()

,customer_unique_id,month,weeks_in_month,quantity,purchases_by_month
0,0000366f3b9a7992bf8c76cfdf3221e2,May,4.428571,1,0.225806
1,0000b849f77a49e4a4ce2b2a4ca5be3f,May,4.428571,1,0.225806
2,0000f46a3911fa3c0805444483337064,March,4.428571,1,0.225806
3,0000f6ccb0745a6a4b88665a16c9f078,October,4.428571,1,0.225806
4,0004aac84e0df4da2b147fca70cf8255,November,4.285714,1,0.233333


In [140]:
mean_per_week=purchases_per_month.groupby(['customer_unique_id','month'])\
    .agg({'purchases_by_month':'mean'})
mean_per_week.head()

,,purchases_by_month
customer_unique_id,month,
0000366f3b9a7992bf8c76cfdf3221e2,May,0.225806
0000b849f77a49e4a4ce2b2a4ca5be3f,May,0.225806
0000f46a3911fa3c0805444483337064,March,0.225806
0000f6ccb0745a6a4b88665a16c9f078,October,0.225806
0004aac84e0df4da2b147fca70cf8255,November,0.233333


In [141]:
#=================================================================================================================TASK5

In [142]:
sns.set(
    font_scale =3,
    style      ="whitegrid",
    rc         ={'figure.figsize':(40,7)}
)


In [143]:
purchases=success_purchases.drop(['month','weeks_in_month','weekday'],axis=1)
purchases.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_pay_success,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,3ce436f183e68e07877b285a838db11a,delivered,2017-09-13 08:59:02,2017-09-13 09:45:35,2017-09-19 18:34:16,2017-09-20 23:43:48,2017-09-29 00:00:00,True,871766c5855e863f6eccc05f988b23cb,28013,campos dos goytacazes,RJ
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,f6dd3ec061db4e3987629fe6b26e5cce,delivered,2017-04-26 10:53:06,2017-04-26 11:05:13,2017-05-04 14:35:00,2017-05-12 16:04:24,2017-05-15 00:00:00,True,eb28e67c4c0b83846050ddfb8a35d051,15775,santa fe do sul,SP
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,6489ae5e4333f3693df5ad4372dab6d3,delivered,2018-01-14 14:33:31,2018-01-14 14:48:30,2018-01-16 12:36:48,2018-01-22 13:19:16,2018-02-05 00:00:00,True,3818d81c6709e39d06b2738a8d3a2474,35661,para de minas,MG
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,d4eb9395c8c0431ee92fce09860c5a06,delivered,2018-08-08 10:00:35,2018-08-08 10:10:18,2018-08-10 13:28:00,2018-08-14 13:32:39,2018-08-20 00:00:00,True,af861d436cfc08b2c2ddefd0ba074622,12952,atibaia,SP
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,58dbd0b2d70206bf40e62cd34e84d795,delivered,2017-02-04 13:57:51,2017-02-04 14:10:13,2017-02-16 09:46:09,2017-03-01 16:42:31,2017-03-17 00:00:00,True,64b576fb70d441e8f1b2d7d446e483c5,13226,varzea paulista,SP


In [144]:
purchases.order_approved_at=purchases.order_approved_at.dt.strftime('%Y-%m')
purchases=purchases.query("order_approved_at != '1970-01'")
purchases.order_approved_at=pd.to_datetime(purchases.order_approved_at)

In [145]:
first_order=purchases.groupby('customer_unique_id',as_index=False)\
    .agg({'order_approved_at':'min'})\
    .rename(columns={'order_approved_at':'first_purchase'})
first_order.first_purchase=first_order.first_purchase.dt.strftime('%Y-%m')
first_order.head()

,customer_unique_id,first_purchase
0,0000366f3b9a7992bf8c76cfdf3221e2,2018-05
1,0000b849f77a49e4a4ce2b2a4ca5be3f,2018-05
2,0000f46a3911fa3c0805444483337064,2017-03
3,0000f6ccb0745a6a4b88665a16c9f078,2017-10
4,0004aac84e0df4da2b147fca70cf8255,2017-11


In [146]:
min_date=first_order.first_purchase.min()
max_date=first_order.first_purchase.max()

In [147]:
period=pd.DataFrame(pd.date_range(min_date,max_date,freq='MS'))
period.columns=['purchase_month']
period

,purchase_month
0,2016-09-01
1,2016-10-01
2,2016-11-01
3,2016-12-01
4,2017-01-01
5,2017-02-01
6,2017-03-01
7,2017-04-01
8,2017-05-01
9,2017-06-01


In [148]:
first_order['key']=1
period['key']=1

In [149]:
template = first_order.merge(period,on='key')[['customer_unique_id','first_purchase','purchase_month']]

template.head()

,customer_unique_id,first_purchase,purchase_month
0,0000366f3b9a7992bf8c76cfdf3221e2,2018-05,2016-09-01
1,0000366f3b9a7992bf8c76cfdf3221e2,2018-05,2016-10-01
2,0000366f3b9a7992bf8c76cfdf3221e2,2018-05,2016-11-01
3,0000366f3b9a7992bf8c76cfdf3221e2,2018-05,2016-12-01
4,0000366f3b9a7992bf8c76cfdf3221e2,2018-05,2017-01-01


In [150]:
template.dtypes

customer_unique_id            object
first_purchase                object
purchase_month        datetime64[ns]
dtype: object

In [151]:
template.first_purchase=pd.to_datetime(template.first_purchase)

In [152]:
template.first_purchase=template.first_purchase.dt.strftime('%Y-%m')

In [153]:
template.purchase_month=template.purchase_month.dt.strftime('%Y-%m')

In [154]:
template.head()

,customer_unique_id,first_purchase,purchase_month
0,0000366f3b9a7992bf8c76cfdf3221e2,2018-05,2016-09
1,0000366f3b9a7992bf8c76cfdf3221e2,2018-05,2016-10
2,0000366f3b9a7992bf8c76cfdf3221e2,2018-05,2016-11
3,0000366f3b9a7992bf8c76cfdf3221e2,2018-05,2016-12
4,0000366f3b9a7992bf8c76cfdf3221e2,2018-05,2017-01


In [155]:
template = template.query("purchase_month >= first_purchase ").reset_index(drop=True)
template.head()

,customer_unique_id,first_purchase,purchase_month
0,0000366f3b9a7992bf8c76cfdf3221e2,2018-05,2018-05
1,0000366f3b9a7992bf8c76cfdf3221e2,2018-05,2018-06
2,0000366f3b9a7992bf8c76cfdf3221e2,2018-05,2018-07
3,0000366f3b9a7992bf8c76cfdf3221e2,2018-05,2018-08
4,0000b849f77a49e4a4ce2b2a4ca5be3f,2018-05,2018-05


In [156]:
money_by_day=purchases.groupby(['customer_unique_id','order_approved_at'],as_index=False)\
    .agg({'price':'sum'})\
    .rename(columns={'order_approved_at':'purchase_month'})

money_by_day.head()

,customer_unique_id,purchase_month,price
0,0000366f3b9a7992bf8c76cfdf3221e2,2018-05-01,129.90
1,0000b849f77a49e4a4ce2b2a4ca5be3f,2018-05-01,18.90
2,0000f46a3911fa3c0805444483337064,2017-03-01,69.00
3,0000f6ccb0745a6a4b88665a16c9f078,2017-10-01,25.99
4,0004aac84e0df4da2b147fca70cf8255,2017-11-01,180.00


In [157]:
money_by_day.purchase_month = money_by_day.purchase_month.dt.strftime('%Y-%m')

In [158]:
money_by_day.head()

,customer_unique_id,purchase_month,price
0,0000366f3b9a7992bf8c76cfdf3221e2,2018-05,129.90
1,0000b849f77a49e4a4ce2b2a4ca5be3f,2018-05,18.90
2,0000f46a3911fa3c0805444483337064,2017-03,69.00
3,0000f6ccb0745a6a4b88665a16c9f078,2017-10,25.99
4,0004aac84e0df4da2b147fca70cf8255,2017-11,180.00


In [159]:
template.dtypes

customer_unique_id    object
first_purchase        object
purchase_month        object
dtype: object

In [160]:
money_by_day.dtypes

customer_unique_id     object
purchase_month         object
price                 float64
dtype: object

In [161]:
final=template.merge(money_by_day, how='left', on=['customer_unique_id','purchase_month'])
final['activity']=(final.price > 0)*1
final=final.drop('price', axis=1)
final.head()

,customer_unique_id,first_purchase,purchase_month,activity
0,0000366f3b9a7992bf8c76cfdf3221e2,2018-05,2018-05,1
1,0000366f3b9a7992bf8c76cfdf3221e2,2018-05,2018-06,0
2,0000366f3b9a7992bf8c76cfdf3221e2,2018-05,2018-07,0
3,0000366f3b9a7992bf8c76cfdf3221e2,2018-05,2018-08,0
4,0000b849f77a49e4a4ce2b2a4ca5be3f,2018-05,2018-05,1


In [162]:
final=final.groupby(['first_purchase','purchase_month'],as_index=False)\
    .agg({'activity':'mean'})
final

,first_purchase,purchase_month,activity
0,2016-09,2016-09,1.000000
1,2016-09,2016-10,0.000000
2,2016-09,2016-11,0.000000
3,2016-09,2016-12,0.000000
4,2016-09,2017-01,0.000000
...,...,...,...
273,2018-06,2018-07,0.003879
274,2018-06,2018-08,0.002698
275,2018-07,2018-07,1.000000
276,2018-07,2018-08,0.005041


In [163]:
heatmap_table=final.pivot(index='first_purchase',columns='purchase_month',values='activity')
heatmap_table

purchase_month,2016-09,2016-10,2016-11,2016-12,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,...,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08
first_purchase,,,,,,,,,,,,,,,,,,,,,
2016-09,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2016-10,NaN,1.0,0.0,0.0,0.0,0.000000,0.000000,0.003257,0.000000,0.000000,...,0.003257,0.000000,0.003257,0.000000,0.003257,0.000000,0.006515,0.006515,0.000000,0.000000
2016-12,NaN,NaN,NaN,1.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2017-01,NaN,NaN,NaN,NaN,1.0,0.005571,0.002786,0.001393,0.004178,0.001393,...,0.002786,0.002786,0.006964,0.004178,0.001393,0.001393,0.002786,0.004178,0.000000,0.001393
2017-02,NaN,NaN,NaN,NaN,NaN,1.000000,0.002368,0.002368,0.001776,0.004144,...,0.001776,0.001184,0.002960,0.001184,0.001776,0.001184,0.000592,0.000592,0.002368,0.000000
2017-03,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.004992,0.003456,0.003456,...,0.003072,0.001152,0.003840,0.001152,0.002304,0.001152,0.001536,0.002304,0.000768,0.001536
2017-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.006050,0.002161,...,0.002593,0.003457,0.001729,0.002161,0.000864,0.000864,0.000432,0.000864,0.000864,0.001729
2017-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.005336,...,0.004212,0.001404,0.002527,0.002808,0.002808,0.003370,0.001966,0.000562,0.001966,0.002527
2017-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,...,0.003521,0.003841,0.002241,0.001280,0.002241,0.003201,0.003841,0.001601,0.001280,0.001921


In [164]:
final_1=final

In [165]:
final_1.first_purchase=pd.to_datetime(final_1.first_purchase)
final_1.purchase_month=pd.to_datetime(final_1.purchase_month)

In [166]:
final_1.dtypes

first_purchase    datetime64[ns]
purchase_month    datetime64[ns]
activity                 float64
dtype: object

In [167]:
final_1.head()

,first_purchase,purchase_month,activity
0,2016-09-01,2016-09-01,1.0
1,2016-09-01,2016-10-01,0.0
2,2016-09-01,2016-11-01,0.0
3,2016-09-01,2016-12-01,0.0
4,2016-09-01,2017-01-01,0.0


In [168]:
final_1=final_1.query("'2017-01-01'<= purchase_month <= '2018-03-01' and '2017-01-01'<= first_purchase <= '2017-12-01'")

In [169]:
heatmap_table_1=final_1.pivot(index='first_purchase',columns='purchase_month',values='activity')


In [170]:
ur_style = (heatmap_table_1
            .style
            .set_caption('User retention by cohort')  # добавляем подпись
            .background_gradient(cmap='viridis')  # раскрашиваем ячейки по столбцам
            .highlight_null('white')  # делаем белый фон для значений NaN
            .format("{:.2%}", na_rep=""))  # числа форматируем как проценты, NaN заменяем на пустоту
ur_style

purchase_month,2017-01-01 00:00:00,2017-02-01 00:00:00,2017-03-01 00:00:00,2017-04-01 00:00:00,2017-05-01 00:00:00,2017-06-01 00:00:00,2017-07-01 00:00:00,2017-08-01 00:00:00,2017-09-01 00:00:00,2017-10-01 00:00:00,2017-11-01 00:00:00,2017-12-01 00:00:00,2018-01-01 00:00:00,2018-02-01 00:00:00,2018-03-01 00:00:00
first_purchase,,,,,,,,,,,,,,,
2017-01-01 00:00:00,100.00%,0.56%,0.28%,0.14%,0.42%,0.14%,0.42%,0.14%,0.14%,0.00%,0.28%,0.28%,0.70%,0.42%,0.14%
2017-02-01 00:00:00,,100.00%,0.24%,0.24%,0.18%,0.41%,0.12%,0.24%,0.18%,0.06%,0.18%,0.12%,0.30%,0.12%,0.18%
2017-03-01 00:00:00,,,100.00%,0.50%,0.35%,0.35%,0.35%,0.15%,0.15%,0.31%,0.31%,0.12%,0.38%,0.12%,0.23%
2017-04-01 00:00:00,,,,100.00%,0.61%,0.22%,0.17%,0.30%,0.26%,0.35%,0.26%,0.35%,0.17%,0.22%,0.09%
2017-05-01 00:00:00,,,,,100.00%,0.53%,0.48%,0.39%,0.31%,0.34%,0.42%,0.14%,0.25%,0.28%,0.28%
2017-06-01 00:00:00,,,,,,100.00%,0.48%,0.35%,0.38%,0.26%,0.35%,0.38%,0.22%,0.13%,0.22%
2017-07-01 00:00:00,,,,,,,100.00%,0.53%,0.34%,0.18%,0.34%,0.18%,0.32%,0.11%,0.18%
2017-08-01 00:00:00,,,,,,,,100.00%,0.67%,0.36%,0.26%,0.38%,0.53%,0.26%,0.26%
2017-09-01 00:00:00,,,,,,,,,100.00%,0.66%,0.49%,0.34%,0.41%,0.27%,0.22%


In [171]:
#Самый высокий показатель retention у пользователей из когорты, сделавшей первую покупку в мае 2015 года

In [172]:
#=================================================================================================================TASK6

In [173]:
purchases.shape

(112635, 19)

In [174]:
purchases.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_pay_success,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,3ce436f183e68e07877b285a838db11a,delivered,2017-09-13 08:59:02,2017-09-01,2017-09-19 18:34:16,2017-09-20 23:43:48,2017-09-29 00:00:00,True,871766c5855e863f6eccc05f988b23cb,28013,campos dos goytacazes,RJ
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,f6dd3ec061db4e3987629fe6b26e5cce,delivered,2017-04-26 10:53:06,2017-04-01,2017-05-04 14:35:00,2017-05-12 16:04:24,2017-05-15 00:00:00,True,eb28e67c4c0b83846050ddfb8a35d051,15775,santa fe do sul,SP
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,6489ae5e4333f3693df5ad4372dab6d3,delivered,2018-01-14 14:33:31,2018-01-01,2018-01-16 12:36:48,2018-01-22 13:19:16,2018-02-05 00:00:00,True,3818d81c6709e39d06b2738a8d3a2474,35661,para de minas,MG
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,d4eb9395c8c0431ee92fce09860c5a06,delivered,2018-08-08 10:00:35,2018-08-01,2018-08-10 13:28:00,2018-08-14 13:32:39,2018-08-20 00:00:00,True,af861d436cfc08b2c2ddefd0ba074622,12952,atibaia,SP
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,58dbd0b2d70206bf40e62cd34e84d795,delivered,2017-02-04 13:57:51,2017-02-01,2017-02-16 09:46:09,2017-03-01 16:42:31,2017-03-17 00:00:00,True,64b576fb70d441e8f1b2d7d446e483c5,13226,varzea paulista,SP


In [175]:
for_segment=purchases[['customer_unique_id','order_id','product_id','order_delivered_customer_date','price']]
for_segment.head()

,customer_unique_id,order_id,product_id,order_delivered_customer_date,price
0,871766c5855e863f6eccc05f988b23cb,00010242fe8c5a6d1ba2dd792cb16214,4244733e06e7ecb4970a6e2683c13e61,2017-09-20 23:43:48,58.90
1,eb28e67c4c0b83846050ddfb8a35d051,00018f77f2f0320c557190d7a144bdd3,e5f2d52b802189ee658865ca93d83a8f,2017-05-12 16:04:24,239.90
2,3818d81c6709e39d06b2738a8d3a2474,000229ec398224ef6ca0657da4fc703e,c777355d18b72b67abbeef9df44fd0fd,2018-01-22 13:19:16,199.00
3,af861d436cfc08b2c2ddefd0ba074622,00024acbcdf0a6daa1e931b038114c75,7634da152a4610f1595efa32f14722fc,2018-08-14 13:32:39,12.99
4,64b576fb70d441e8f1b2d7d446e483c5,00042b26cf59d7ce69dfabb4e55b4fd9,ac6c3623068f30de03045865e4e10089,2017-03-01 16:42:31,199.90


In [176]:
for_segment.order_delivered_customer_date=pd.to_datetime(for_segment.order_delivered_customer_date)

/opt/tljh/user/lib/python3.7/site-packages/pandas/core/generic.py:5165: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [177]:
for_segment.dtypes

customer_unique_id                       object
order_id                                 object
product_id                               object
order_delivered_customer_date    datetime64[ns]
price                                   float64
dtype: object

In [178]:
for_segment.order_delivered_customer_date=for_segment.order_delivered_customer_date.dt.strftime('%Y-%m-%d')

In [179]:
for_segment.head()

,customer_unique_id,order_id,product_id,order_delivered_customer_date,price
0,871766c5855e863f6eccc05f988b23cb,00010242fe8c5a6d1ba2dd792cb16214,4244733e06e7ecb4970a6e2683c13e61,2017-09-20,58.90
1,eb28e67c4c0b83846050ddfb8a35d051,00018f77f2f0320c557190d7a144bdd3,e5f2d52b802189ee658865ca93d83a8f,2017-05-12,239.90
2,3818d81c6709e39d06b2738a8d3a2474,000229ec398224ef6ca0657da4fc703e,c777355d18b72b67abbeef9df44fd0fd,2018-01-22,199.00
3,af861d436cfc08b2c2ddefd0ba074622,00024acbcdf0a6daa1e931b038114c75,7634da152a4610f1595efa32f14722fc,2018-08-14,12.99
4,64b576fb70d441e8f1b2d7d446e483c5,00042b26cf59d7ce69dfabb4e55b4fd9,ac6c3623068f30de03045865e4e10089,2017-03-01,199.90


In [180]:
for_segment.order_delivered_customer_date=pd.to_datetime(for_segment.order_delivered_customer_date)

/opt/tljh/user/lib/python3.7/site-packages/pandas/core/generic.py:5165: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [181]:
for_segment.dtypes

customer_unique_id                       object
order_id                                 object
product_id                               object
order_delivered_customer_date    datetime64[ns]
price                                   float64
dtype: object

In [182]:
today=for_segment.order_delivered_customer_date.max() + dt.timedelta(days=1)
today

Timestamp('2018-10-18 00:00:00')

In [183]:
last_purchase=for_segment.groupby('customer_unique_id',as_index=False)\
    .agg({'order_delivered_customer_date':'max'})\
    .sort_values('order_delivered_customer_date', ascending=False)\
    .drop_duplicates(subset='customer_unique_id')\
    .rename(columns={'order_delivered_customer_date':'last_purchase'})
last_purchase.head()

,customer_unique_id,last_purchase
93938,fc0bcca3d8d90a812b707bb8e45069d6,2018-10-17
13168,232422c2db1f7c69daf3b0f0421ebb43,2018-10-11
28297,4bc530761c1b59d33d7c6cb640d3ee78,2018-10-02
22049,3ae68a873484afec224de126a94c5974,2018-09-27
75378,c9f2b70eaddf459267a9de373aaf743b,2018-09-25


In [184]:
for_segment=for_segment.merge(last_purchase, how='inner', on='customer_unique_id')
for_segment.head()

,customer_unique_id,order_id,product_id,order_delivered_customer_date,price,last_purchase
0,871766c5855e863f6eccc05f988b23cb,00010242fe8c5a6d1ba2dd792cb16214,4244733e06e7ecb4970a6e2683c13e61,2017-09-20,58.90,2017-09-20
1,eb28e67c4c0b83846050ddfb8a35d051,00018f77f2f0320c557190d7a144bdd3,e5f2d52b802189ee658865ca93d83a8f,2017-05-12,239.90,2017-09-25
2,eb28e67c4c0b83846050ddfb8a35d051,c105be01f21c9df6a2579f59b95cbcd9,d2f5484cbffe4ca766301b21ab9246dd,2017-09-25,12.88,2017-09-25
3,3818d81c6709e39d06b2738a8d3a2474,000229ec398224ef6ca0657da4fc703e,c777355d18b72b67abbeef9df44fd0fd,2018-01-22,199.00,2018-01-22
4,af861d436cfc08b2c2ddefd0ba074622,00024acbcdf0a6daa1e931b038114c75,7634da152a4610f1595efa32f14722fc,2018-08-14,12.99,2018-08-14


In [185]:
for_segment['today']=today
for_segment.head()

,customer_unique_id,order_id,product_id,order_delivered_customer_date,price,last_purchase,today
0,871766c5855e863f6eccc05f988b23cb,00010242fe8c5a6d1ba2dd792cb16214,4244733e06e7ecb4970a6e2683c13e61,2017-09-20,58.90,2017-09-20,2018-10-18
1,eb28e67c4c0b83846050ddfb8a35d051,00018f77f2f0320c557190d7a144bdd3,e5f2d52b802189ee658865ca93d83a8f,2017-05-12,239.90,2017-09-25,2018-10-18
2,eb28e67c4c0b83846050ddfb8a35d051,c105be01f21c9df6a2579f59b95cbcd9,d2f5484cbffe4ca766301b21ab9246dd,2017-09-25,12.88,2017-09-25,2018-10-18
3,3818d81c6709e39d06b2738a8d3a2474,000229ec398224ef6ca0657da4fc703e,c777355d18b72b67abbeef9df44fd0fd,2018-01-22,199.00,2018-01-22,2018-10-18
4,af861d436cfc08b2c2ddefd0ba074622,00024acbcdf0a6daa1e931b038114c75,7634da152a4610f1595efa32f14722fc,2018-08-14,12.99,2018-08-14,2018-10-18


In [186]:
for_segment['for_R']=for_segment.today-for_segment.last_purchase
for_segment.head()

,customer_unique_id,order_id,product_id,order_delivered_customer_date,price,last_purchase,today,for_R
0,871766c5855e863f6eccc05f988b23cb,00010242fe8c5a6d1ba2dd792cb16214,4244733e06e7ecb4970a6e2683c13e61,2017-09-20,58.90,2017-09-20,2018-10-18,393 days
1,eb28e67c4c0b83846050ddfb8a35d051,00018f77f2f0320c557190d7a144bdd3,e5f2d52b802189ee658865ca93d83a8f,2017-05-12,239.90,2017-09-25,2018-10-18,388 days
2,eb28e67c4c0b83846050ddfb8a35d051,c105be01f21c9df6a2579f59b95cbcd9,d2f5484cbffe4ca766301b21ab9246dd,2017-09-25,12.88,2017-09-25,2018-10-18,388 days
3,3818d81c6709e39d06b2738a8d3a2474,000229ec398224ef6ca0657da4fc703e,c777355d18b72b67abbeef9df44fd0fd,2018-01-22,199.00,2018-01-22,2018-10-18,269 days
4,af861d436cfc08b2c2ddefd0ba074622,00024acbcdf0a6daa1e931b038114c75,7634da152a4610f1595efa32f14722fc,2018-08-14,12.99,2018-08-14,2018-10-18,65 days


In [187]:
purchases.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_pay_success,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,3ce436f183e68e07877b285a838db11a,delivered,2017-09-13 08:59:02,2017-09-01,2017-09-19 18:34:16,2017-09-20 23:43:48,2017-09-29 00:00:00,True,871766c5855e863f6eccc05f988b23cb,28013,campos dos goytacazes,RJ
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,f6dd3ec061db4e3987629fe6b26e5cce,delivered,2017-04-26 10:53:06,2017-04-01,2017-05-04 14:35:00,2017-05-12 16:04:24,2017-05-15 00:00:00,True,eb28e67c4c0b83846050ddfb8a35d051,15775,santa fe do sul,SP
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,6489ae5e4333f3693df5ad4372dab6d3,delivered,2018-01-14 14:33:31,2018-01-01,2018-01-16 12:36:48,2018-01-22 13:19:16,2018-02-05 00:00:00,True,3818d81c6709e39d06b2738a8d3a2474,35661,para de minas,MG
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,d4eb9395c8c0431ee92fce09860c5a06,delivered,2018-08-08 10:00:35,2018-08-01,2018-08-10 13:28:00,2018-08-14 13:32:39,2018-08-20 00:00:00,True,af861d436cfc08b2c2ddefd0ba074622,12952,atibaia,SP
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,58dbd0b2d70206bf40e62cd34e84d795,delivered,2017-02-04 13:57:51,2017-02-01,2017-02-16 09:46:09,2017-03-01 16:42:31,2017-03-17 00:00:00,True,64b576fb70d441e8f1b2d7d446e483c5,13226,varzea paulista,SP


In [188]:
for_F=purchases.groupby('customer_unique_id',as_index=False)\
    .agg({'order_id':'count'})\
    .rename(columns={'order_id':'for_F'})
for_F.head()

,customer_unique_id,for_F
0,0000366f3b9a7992bf8c76cfdf3221e2,1
1,0000b849f77a49e4a4ce2b2a4ca5be3f,1
2,0000f46a3911fa3c0805444483337064,1
3,0000f6ccb0745a6a4b88665a16c9f078,1
4,0004aac84e0df4da2b147fca70cf8255,1


In [189]:
for_M=purchases.groupby('customer_unique_id',as_index=False)\
    .agg({'price':'sum'})\
    .rename(columns={'price':'for_M'})
for_M.head()

,customer_unique_id,for_M
0,0000366f3b9a7992bf8c76cfdf3221e2,129.90
1,0000b849f77a49e4a4ce2b2a4ca5be3f,18.90
2,0000f46a3911fa3c0805444483337064,69.00
3,0000f6ccb0745a6a4b88665a16c9f078,25.99
4,0004aac84e0df4da2b147fca70cf8255,180.00


In [190]:
for_segment=for_segment.merge(for_F, on='customer_unique_id').merge(for_M, on='customer_unique_id')
for_segment.head()

,customer_unique_id,order_id,product_id,order_delivered_customer_date,price,last_purchase,today,for_R,for_F,for_M
0,871766c5855e863f6eccc05f988b23cb,00010242fe8c5a6d1ba2dd792cb16214,4244733e06e7ecb4970a6e2683c13e61,2017-09-20,58.90,2017-09-20,2018-10-18,393 days,1,58.90
1,eb28e67c4c0b83846050ddfb8a35d051,00018f77f2f0320c557190d7a144bdd3,e5f2d52b802189ee658865ca93d83a8f,2017-05-12,239.90,2017-09-25,2018-10-18,388 days,2,252.78
2,eb28e67c4c0b83846050ddfb8a35d051,c105be01f21c9df6a2579f59b95cbcd9,d2f5484cbffe4ca766301b21ab9246dd,2017-09-25,12.88,2017-09-25,2018-10-18,388 days,2,252.78
3,3818d81c6709e39d06b2738a8d3a2474,000229ec398224ef6ca0657da4fc703e,c777355d18b72b67abbeef9df44fd0fd,2018-01-22,199.00,2018-01-22,2018-10-18,269 days,1,199.00
4,af861d436cfc08b2c2ddefd0ba074622,00024acbcdf0a6daa1e931b038114c75,7634da152a4610f1595efa32f14722fc,2018-08-14,12.99,2018-08-14,2018-10-18,65 days,1,12.99


In [191]:
for_segment=for_segment[['customer_unique_id','for_R','for_F','for_M']]
for_segment.for_R = for_segment.for_R.astype(str)
for_segment.head()

,customer_unique_id,for_R,for_F,for_M
0,871766c5855e863f6eccc05f988b23cb,393 days,1,58.90
1,eb28e67c4c0b83846050ddfb8a35d051,388 days,2,252.78
2,eb28e67c4c0b83846050ddfb8a35d051,388 days,2,252.78
3,3818d81c6709e39d06b2738a8d3a2474,269 days,1,199.00
4,af861d436cfc08b2c2ddefd0ba074622,65 days,1,12.99


In [192]:
for_segment.for_R=for_segment.for_R.apply(lambda x: x.split(' ')[0])
for_segment.for_M=for_segment.for_M.round()
for_segment.head()

,customer_unique_id,for_R,for_F,for_M
0,871766c5855e863f6eccc05f988b23cb,393,1,59.0
1,eb28e67c4c0b83846050ddfb8a35d051,388,2,253.0
2,eb28e67c4c0b83846050ddfb8a35d051,388,2,253.0
3,3818d81c6709e39d06b2738a8d3a2474,269,1,199.0
4,af861d436cfc08b2c2ddefd0ba074622,65,1,13.0


In [193]:
for_segment.head()

,customer_unique_id,for_R,for_F,for_M
0,871766c5855e863f6eccc05f988b23cb,393,1,59.0
1,eb28e67c4c0b83846050ddfb8a35d051,388,2,253.0
2,eb28e67c4c0b83846050ddfb8a35d051,388,2,253.0
3,3818d81c6709e39d06b2738a8d3a2474,269,1,199.0
4,af861d436cfc08b2c2ddefd0ba074622,65,1,13.0


In [199]:
for_segment=for_segment.query("for_R != 'NaT'")

In [200]:
for_segment.dtypes

customer_unique_id     object
for_R                  object
for_F                   int64
for_M                 float64
dtype: object

In [201]:
for_segment['for_R'] = for_segment['for_R'].astype(int)

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [202]:
for_segment.dtypes

customer_unique_id     object
for_R                   int64
for_F                   int64
for_M                 float64
dtype: object

In [204]:
level=[0.25,0.5,0.75]
quantiles = for_segment[['for_R', 'for_F', 'for_M']].quantile(level).to_dict()
quantiles

{'for_R': {0.25: 154.0, 0.5: 253.0, 0.75: 381.0},
 'for_F': {0.25: 1.0, 0.5: 1.0, 0.75: 2.0},
 'for_M': {0.25: 50.0, 0.5: 99.0, 0.75: 178.0}}

In [206]:
intervals_R=[0]
for i in range(len(quantiles['for_R'])): 
    intervals_R.append(quantiles['for_R'][level[i]])
intervals_R.append(for_segment.for_R.max())
for_segment['R']=pd.cut(for_segment.for_R, intervals_R, labels=['4', '3', '2', '1'])

In [ ]:
#Выведенные границы интервалов для "for_F" не подходят для корректной сегментации колонки, поэтому добавлю самостоятельно

In [213]:
for_segment.for_F.unique()

array([ 1,  2,  3,  4,  5,  7,  6, 24,  8, 10,  9, 14, 16, 11, 12, 20, 13,
       18, 15, 21])

In [214]:
intervals_F=[0,6,12,18,24]
for_segment['F']=pd.cut(for_segment.for_F, intervals_F, labels=['1', '2', '3', '4'])

In [221]:
intervals_M=[0]
for i in range(len(quantiles['for_M'])): 
    intervals_M.append(quantiles['for_M'][level[i]])
intervals_M.append(for_segment.for_M.max())
for_segment['M']=pd.cut(for_segment.for_M, intervals_M, labels=['1', '2', '3', '4'])

In [223]:
for_segment=for_segment.astype({'R':'string', 'F':'string', 'M':'string'})
for_segment['RFM'] = for_segment['R'] + for_segment['F'] + for_segment['M']
for_segment.head()

,customer_unique_id,for_R,for_F,for_M,R,F,M,RFM
0,871766c5855e863f6eccc05f988b23cb,393,1,59.0,1,1,2,112
1,eb28e67c4c0b83846050ddfb8a35d051,388,2,253.0,1,1,4,114
2,eb28e67c4c0b83846050ddfb8a35d051,388,2,253.0,1,1,4,114
3,3818d81c6709e39d06b2738a8d3a2474,269,1,199.0,2,1,4,214
4,af861d436cfc08b2c2ddefd0ba074622,65,1,13.0,4,1,1,411


In [234]:
for_segment['Segment'] = for_segment['R'].map(str) + for_segment['F'].map(str) + for_segment['M'].map(str)
segment_map = {
    r'[1-2][1-2][1-2]': 'бездействующие', # покупали давно, мало заказов, маленькая выручка 
    r'[1-2][1-2][3-4]': 'в зоне риска', # покупали давно, мало заказов, средняя выручка 
    r'[1-2][1-4]4': 'не можем потерять', # покупали давно, не мало заказов, высокая выручка 
    r'3[1-2][1-2]': 'спящие', # покупали относительно недавно, мало заказов, маленькая выручка 
    r'3[1-2]3': 'требуют внимание', # покупали относительно недавно, мало заказов, средняя выручка 
    r'[3-4][1-4][3-4]': 'лояльные пользователи', # покупали относительно недавно, не мало заказов, высокая выручка 
    r'4[1-4]1': 'многообещающие', # покупали недавно, не мало заказов, маленькая выручка 
    r'4[1-4]1': 'новые пользователи', # покупали недавно, не мало заказов, маленькая выручка 
    r'[3-4][1-4][2-3]': 'потенциально-лояльные', # покупали недавно, не мало заказов, средняя выручка 
    r'4[1-4][3-4]': 'чемпионы'} # покупали недавно, не мало заказов, высокая выручка 

# заменим ранги на названия сегментов
for_segment['Segment'] = for_segment['Segment'].replace(segment_map, regex=True)

In [235]:
for_segment.head()

,customer_unique_id,for_R,for_F,for_M,R,F,M,RFM,Segment
0,871766c5855e863f6eccc05f988b23cb,393,1,59.0,1,1,2,112,бездействующие
1,eb28e67c4c0b83846050ddfb8a35d051,388,2,253.0,1,1,4,114,в зоне риска
2,eb28e67c4c0b83846050ddfb8a35d051,388,2,253.0,1,1,4,114,в зоне риска
3,3818d81c6709e39d06b2738a8d3a2474,269,1,199.0,2,1,4,214,в зоне риска
4,af861d436cfc08b2c2ddefd0ba074622,65,1,13.0,4,1,1,411,новые пользователи
